In [1]:
from DB_System.communication import DB_Handle, DataHandle
from BackTesting_System.backtesting import EconomicIndicators, BackTesting, Utils

In [2]:
import pandas as pd

In [3]:
def make_signal(df):
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        '''buy 조건 '''
        con1=(df.loc[idx[0]:idx[-1],'5_mom'].shift(2)<100)&(df.loc[idx[0]:idx[-1],'5_mom'].shift(2)>=95)
        con2=(df.loc[idx[0]:idx[-1],'5_mom'].shift(1)>=100)&(df.loc[idx[0]:idx[-1],'5_mom'].shift(1)<110)
        con3=(df.loc[idx[0]:idx[-1],'5_mom']>=df.loc[idx[0]:idx[-1],'5_mom'].shift(1))&(df.loc[idx[0]:idx[-1],'5_mom']<110)
        con4=(df.loc[idx[0]:idx[-1],'누적거래대금']>=10000000000)
        df.loc[idx[0]:idx[-1], 'buy'] = con1&con2&con3&con4
        '''sell 조건 '''
        con5=(df.loc[idx[0]:idx[-1],'buy'].shift(5)==True) #5일뒤 매도하는 조건
        df.loc[idx[0]:idx[-1], 'sell'] = con5
    return df

In [4]:
# dbhandler = DB_Handle(db_name='kospi_fin')
dh = DataHandle(db_name='kospi_fin')
ec = EconomicIndicators()
bt = BackTesting()
ut = Utils()

In [5]:
# df = dh.get_data_for_pjt(period='all', companys=['포스코케미칼', '미래에셋생명', '현대로템', '한진칼'])
# df['종목명'] = df['코드'].apply(lambda x: dh.KOSPI_CODES[x])

In [6]:
df = pd.read_csv('../result.csv')
df['코드'] = df['코드'].apply(lambda x: str(x).zfill(6))

In [7]:
df_test = make_signal(df)

In [8]:
bt.MONEY = 2000000

In [9]:
bt.buy_sell(df=df_test, buy='buy', sell='sell')

In [10]:
bt.ASSET

{'006340': 3,
 '006360': 6,
 '005390': 3,
 '000660': 3,
 '010140': 6,
 '044380': 3,
 '033240': 3,
 '047040': 3,
 '024890': 3,
 '017670': 3,
 '267260': 3,
 '114090': 3,
 '103140': 3,
 '004170': 3}

In [11]:
bt.JOUNAL['종목코드'].unique()

array(['089590', '014680', '008700', '000660', '003060', '014160',
       '207940', '086790', '034220', '024110', '000990', '005930',
       '267250', '086280', '073240', '009540', '004020', '010620',
       '178920', '085310', '066570', '267270', '042660', '030000',
       '010140', '004770', '017180', '010060', '109070', '023530',
       '000720', '001340', '097950', '204320', '012280', '016360',
       '000270', '036420', '005940', '105560', '003220', '006800',
       '003550', '058850', '011070', '008560', '005070', '004800',
       '018250', '114090', '020560', '161890', '051600', '090430',
       '035250', '011200', '000880', '009240', '028260', '004170',
       '019170', '145720', '035720', '037560', '078930', '009150',
       '028050', '047810', '018880', '005420', '192820', '012450',
       '006400', '007070', '009180', '009470', '139480', '047040',
       '006260', '009830', '003490', '014580', '008250', '128940',
       '000120', '248170', '081660', '093370', '017670', '0810

In [175]:
bt.JOUNAL[bt.JOUNAL['종목코드'] == '010140']

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
45,2018-03-08,010140,7825,3,33.0,매수
55,2018-03-15,010140,7966,3,34.0,매도
171,2018-05-24,010140,7494,3,32.0,매수
183,2018-05-31,010140,7069,3,30.0,매도
295,2018-08-20,010140,6597,3,28.0,매수
308,2018-08-27,010140,6607,3,28.0,매도
536,2019-02-15,010140,8268,3,35.0,매수
542,2019-02-21,010140,8787,3,37.0,매수
547,2019-02-22,010140,8778,6,74.0,매도
840,2019-11-08,010140,6862,3,29.0,매수


In [16]:
bt.update_information()

In [17]:
ut.making_mt_plot(handle_data=df_test, code='000100', jounal=bt.JOUNAL)

2018-12-19 2018-12-27
2019-01-16 2019-01-23
2020-03-02 2020-03-09
2020-04-09 2020-04-17
2020-04-23 2020-05-04


In [208]:
from copy import deepcopy
dates = []
buf = []
for _, row in bt.JOUNAL[bt.JOUNAL['종목코드'] == '010140'].iterrows():
    print(row['날짜'], row['매매구분'])
    if len(buf) == 2:
        duf_copy = deepcopy(buf)
        dates.append(duf_copy)
        print(dates)
        buf.clear()
        
    if not buf:
        if row['매매구분'] == '매수':
            buf.append(row['날짜'])
            print(buf)
    if row['매매구분'] == '매도':
        buf.append(row['날짜'])
        print(buf)
    

2018-03-08 매수
['2018-03-08']
2018-03-15 매도
['2018-03-08', '2018-03-15']
2018-05-24 매수
[['2018-03-08', '2018-03-15']]
['2018-05-24']
2018-05-31 매도
['2018-05-24', '2018-05-31']
2018-08-20 매수
[['2018-03-08', '2018-03-15'], ['2018-05-24', '2018-05-31']]
['2018-08-20']
2018-08-27 매도
['2018-08-20', '2018-08-27']
2019-02-15 매수
[['2018-03-08', '2018-03-15'], ['2018-05-24', '2018-05-31'], ['2018-08-20', '2018-08-27']]
['2019-02-15']
2019-02-21 매수
2019-02-22 매도
['2019-02-15', '2019-02-22']
2019-11-08 매수
[['2018-03-08', '2018-03-15'], ['2018-05-24', '2018-05-31'], ['2018-08-20', '2018-08-27'], ['2019-02-15', '2019-02-22']]
['2019-11-08']
2019-11-15 매도
['2019-11-08', '2019-11-15']
2020-05-20 매수
[['2018-03-08', '2018-03-15'], ['2018-05-24', '2018-05-31'], ['2018-08-20', '2018-08-27'], ['2019-02-15', '2019-02-22'], ['2019-11-08', '2019-11-15']]
['2020-05-20']
2020-05-27 매도
['2020-05-20', '2020-05-27']
2020-11-05 매수
[['2018-03-08', '2018-03-15'], ['2018-05-24', '2018-05-31'], ['2018-08-20', '2018-08-

In [149]:
a = list(bt.JOUNAL[(bt.JOUNAL['종목코드'] == '010140') & (bt.JOUNAL['매매구분'] == '매수')][['날짜', '매매구분']].itertuples(index=False, name=None))

In [150]:
b = list(bt.JOUNAL[(bt.JOUNAL['종목코드'] == '010140') & (bt.JOUNAL['매매구분'] == '매도')][['날짜', '매매구분']].itertuples(index=False, name=None))

In [151]:
# a = bt.JOUNAL[(bt.JOUNAL['종목코드'] == '034220') & (bt.JOUNAL['매매구분'] == '매수')][['날짜', '매매구분']].set_index('날짜').to_()['매매구분']
# b = bt.JOUNAL[(bt.JOUNAL['종목코드'] == '139130') & (bt.JOUNAL['매매구분'] == '매도')][['날짜', '매매구분']].set_index('날짜').to_dict()['매매구분']

In [12]:
start = pd.to_datetime('19000101')
end = pd.to_datetime('20230115')
code = '377740'

In [13]:
condition = (df['날짜'] > start) & (df['날짜'] < end) & (df['코드'] == code)

In [42]:
df[condition].max()['종가']

10650

In [4]:
sql = f'''select * from 제주항공 '''

In [5]:
res = dbhandler.fetch_base(sql)
res = datahandler(res)

In [6]:
res

,코드,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금
0,89590,2015-11-06,36785,37856,39385,35255,3144622,117995303950
1,89590,2015-11-09,34414,37244,37588,33994,459701,16175815800
2,89590,2015-11-10,32693,33649,35791,32693,341823,11608062150
3,89590,2015-11-11,34185,33114,34988,32885,361457,12348714700
4,89590,2015-11-12,34950,34873,35485,34147,207591,7280780950
...,...,...,...,...,...,...,...,...
1772,89590,2023-01-19,16400,16350,16600,15950,666623,10933975350
1773,89590,2023-01-20,16750,16350,16950,16100,661493,11004799900
1774,89590,2023-01-25,16190,16820,16900,15850,1137683,18475949590
1775,89590,2023-01-26,16070,16000,16300,15900,513798,8255311640


In [12]:
from pykrx import stock
stock.get_index_fundamental("20210101", "20210130", "1001")

,종가,등락률,PER,선행PER,PBR,배당수익률
날짜,,,,,,
2021-01-04,2944.45,2.47,30.200001,0.0,1.19,1.44
2021-01-05,2990.57,1.57,30.680000,0.0,1.20,1.42
2021-01-06,2968.21,-0.75,30.450001,0.0,1.20,1.43
2021-01-07,3031.68,2.14,31.110001,0.0,1.22,1.40
2021-01-08,3152.18,3.97,32.349998,0.0,1.27,1.35
2021-01-11,3148.45,-0.12,32.310001,0.0,1.27,1.35
2021-01-12,3125.95,-0.71,32.080002,0.0,1.26,1.36
2021-01-13,3148.29,0.71,32.310001,0.0,1.27,1.35
2021-01-14,3149.93,0.05,32.330002,0.0,1.27,1.35


In [ ]:
dfs = stock.get_index_fundamental(start,end,code)

In [13]:
import pandas as pd

In [16]:
dbhandler.DB_NAME[:-4]

'kospi'

In [ ]:
pd.DataFrame()